In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.extend([ '../', '../../'])
import torch
import torch.nn as nn
import torch.optim as optim
from data import input_data
import dataloader as dl
from args import Args
from model import Het_En, Classifier, EdgePredictor, Het_classify
from smote import oversample
from train import train_smote, test_smote

# Set device to GPU if available, else use CPU
args = Args()
args.aminer_train()
torch.cuda.empty_cache()

Current device: NVIDIA A100 80GB PCIe


In [2]:
# data = input_data(args) 
# torch.save(data, 'data.pt')
data = torch.load('../../data/aminer/am_data.pt')
print(data)

HeteroData(
  a={
    num_nodes=20171,
    y=[20171]
  },
  p={ num_nodes=13250 },
  v={ num_nodes=18 },
  p_title_embed={ x=[13250, 128] },
  p_abstract_embed={ x=[13250, 128] },
  p_net_embed={ x=[13250, 128] },
  p_a_net_embed={ x=[13250, 128] },
  p_p_net_embed={ x=[13250, 128] },
  p_v_net_embed={ x=[13250, 128] },
  a_net_embed={ x=[20171, 128] },
  a_text_embed={ x=[20171, 128] },
  v_net_embed={ x=[18, 128] },
  v_text_embed={ x=[18, 128] },
  (a, walk, a)={ edge_index=[2, 99480] },
  (a, walk, p)={ edge_index=[2, 86444] },
  (a, walk, v)={ edge_index=[2, 31716] },
  (p, walk, a)={ edge_index=[2, 65534] },
  (p, walk, p)={ edge_index=[2, 66174] },
  (p, walk, v)={ edge_index=[2, 20838] },
  (v, walk, a)={ edge_index=[2, 90] },
  (v, walk, p)={ edge_index=[2, 90] },
  (v, walk, v)={ edge_index=[2, 27] }
)


In [3]:
print(data['a','walk','p'].edge_index)
device = args.device
data = data.to(device)

# Send all x tensors to the device
data['p_title_embed']['x'] = data['p_title_embed']['x'].to(device)
data['p_abstract_embed']['x'] = data['p_abstract_embed']['x'].to(device)
data['p_net_embed']['x'] = data['p_net_embed']['x'].to(device)
data['p_a_net_embed']['x'] = data['p_a_net_embed']['x'].to(device)
data['p_p_net_embed']['x'] = data['p_p_net_embed']['x'].to(device)
data['p_v_net_embed']['x'] = data['p_v_net_embed']['x'].to(device)
data['a_net_embed']['x'] = data['a_net_embed']['x'].to(device)
data['a_text_embed']['x'] = data['a_text_embed']['x'].to(device)
data['v_net_embed']['x'] = data['v_net_embed']['x'].to(device)
data['v_text_embed']['x'] = data['v_text_embed']['x'].to(device)

# Send all y tensors to the device
data['a']['y'] = data['a']['y'].to(device)

# Send all edge_index tensors to the device
data['a', 'walk', 'a']['edge_index'] = data['a', 'walk', 'a']['edge_index'].to(device)
data['a', 'walk', 'p']['edge_index'] = data['a', 'walk', 'p']['edge_index'].to(device)
data['a', 'walk', 'v']['edge_index'] = data['a', 'walk', 'v']['edge_index'].to(device)
data['p', 'walk', 'a']['edge_index'] = data['p', 'walk', 'a']['edge_index'].to(device)
data['p', 'walk', 'p']['edge_index'] = data['p', 'walk', 'p']['edge_index'].to(device)
data['p', 'walk', 'v']['edge_index'] = data['p', 'walk', 'v']['edge_index'].to(device)
data['v', 'walk', 'a']['edge_index'] = data['v', 'walk', 'a']['edge_index'].to(device)
data['v', 'walk', 'p']['edge_index'] = data['v', 'walk', 'p']['edge_index'].to(device)
data['v', 'walk', 'v']['edge_index'] = data['v', 'walk', 'v']['edge_index'].to(device)

edge_indices = [ data['a', 'walk', 'a'].edge_index, data['a', 'walk', 'p'].edge_index, data['a', 'walk', 'v'].edge_index ]

tensor([[    0,     0,     0,  ..., 20170, 20170, 20170],
        [10523, 10442, 10329,  ...,  4378, 12456,  6612]])


In [4]:
c_train_num = dl.train_num(data['a'].y, args.im_class_num, args.class_sample_num, args.im_ratio)
print(c_train_num, sum(c_train_num))
train_idx, val_idx, test_idx, c_num_mat = dl.segregate(data['a'].y, c_train_num)
print("train_idx: ", train_idx, len(train_idx))
print("val_idx: ", val_idx, len(val_idx))
print("test_idx: ", test_idx, len(test_idx))
# print(c_num_mat)

[50, 20, 50, 50] 170
0 335
1 202
2 299
3 289
train_idx:  [13333, 2296, 13577, 14004, 16803, 13893, 13345, 10032, 4649, 17149, 11915, 18862, 68, 13137, 19625, 12698, 6632, 18805, 11992, 13965, 17308, 2741, 18642, 10131, 15925, 9727, 12634, 658, 7503, 1313, 2090, 10925, 17285, 1478, 8711, 5233, 19475, 6960, 9673, 19229, 4172, 13231, 4945, 15795, 9279, 4572, 18584, 1307, 10442, 34, 15341, 1294, 14642, 5946, 16458, 19549, 4325, 10368, 17118, 174, 8410, 3426, 8419, 11422, 10142, 8394, 12380, 7731, 16115, 18182, 18214, 1268, 18747, 13984, 5771, 20158, 19706, 9112, 15791, 12035, 4270, 12461, 10813, 7053, 11434, 950, 11408, 1811, 12236, 575, 15080, 18855, 15814, 16191, 4742, 7513, 11727, 12172, 8059, 18755, 9995, 8366, 9990, 7808, 14500, 5404, 10435, 3968, 7325, 15679, 3627, 7764, 17126, 16664, 14313, 16232, 866, 18885, 18466, 11532, 16931, 17368, 9699, 16654, 14707, 18983, 12315, 1398, 12008, 7076, 4095, 16853, 5820, 3232, 15848, 7953, 20170, 4743, 16522, 6716, 1296, 10961, 14625, 17939, 1763

In [5]:
# os_mode = 'up'
# portion = sum(c_train_num)/(4*c_train_num[1])
# if os_mode != 'reweight' and os_mode != 'no':
#     if portion >= 1:
#         args.node_dim[0] = args.A_n + (portion-1)*c_train_num[1] + int((portion - int(portion))*c_train_num[1])
#     else:
#         args.node_dim[0] = args.A_n + int((portion - int(portion))*c_train_num[1])

# args.node_dim[0] += 22

In [6]:
encoder = Het_En(args.embed_dim, args.dropout)
classifier = Het_classify(args.embed_dim, args.nclass, args.dropout)
decoder_a = EdgePredictor(args.embed_dim)
decoder_p = EdgePredictor(args.embed_dim)
decoder_v = EdgePredictor(args.embed_dim)
decoder_list = [decoder_a, decoder_p, decoder_v]
#print(features.shape)
encoder.to(device)
classifier.to(device)
for decoder in decoder_list:
    decoder.to(device)

## Training Part

In [7]:
torch.cuda.empty_cache()
train_smote(data, edge_indices, encoder, classifier, decoder_list, train_idx, val_idx, test_idx, args, os_mode = 'up', train_mode = '')

Epoch [1/500], Loss: 1.3823, Accuracy: 0.2708, Edge Accuracy: []
Class 0: AUC-ROC- 0.5500, F1 Score- 0.0000; Class 1: AUC-ROC- 0.6081, F1 Score- 0.3605; Class 2: AUC-ROC- 0.5970, F1 Score- 0.0000; Class 3: AUC-ROC- 0.5261, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.5703; Macro-Average F1 Score: 0.0901
Validation Loss: 1.3788, Validation Accuracy: 0.2500, Validation Edge Accuracy: []
Class 0: AUC-ROC- 0.8231, F1 Score- 0.0000; Class 1: AUC-ROC- 0.7809, F1 Score- 0.4000; Class 2: AUC-ROC- 0.5876, F1 Score- 0.0000; Class 3: AUC-ROC- 0.7059, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.7244; Macro-Average F1 Score: 0.1000
Test Loss: 1.3801, Test Accuracy: 0.2525, Test Edge Accuracy: []
Class 0: AUC-ROC- 0.7382, F1 Score- 0.0000; Class 1: AUC-ROC- 0.8641, F1 Score- 0.4000; Class 2: AUC-ROC- 0.5868, F1 Score- 0.0000; Class 3: AUC-ROC- 0.6401, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.7073; Macro-Average F1 Score: 0.1000

Epoch [2/500], Loss: 1.3785, Accuracy: 0.2656, Edge Accuracy: []
Class 0

In [8]:
test_smote(data, edge_indices, encoder, classifier, decoder_list, test_idx, args =args, dataset = 'Test', os_mode = 'no', train_mode = '')

Test Loss: 0.8432, Test Accuracy: 0.9625, Test Edge Accuracy: []
Class 0: AUC-ROC- 0.9994, F1 Score- 0.9802; Class 1: AUC-ROC- 0.9975, F1 Score- 0.9849; Class 2: AUC-ROC- 0.9960, F1 Score- 0.9296; Class 3: AUC-ROC- 0.9836, F1 Score- 0.9641; Macro-Average AUC-ROC: 0.9941; Macro-Average F1 Score: 0.9647


(0.9625, 0.9941416666666667, 0.9647006679522137)